https://www.nitrc.org/plugins/mwiki/index.php/neurobureau:AthenaPipeline

In [1]:
import os
root = os.curdir

#provide dataset directory here
dataset_dir = r"D:\All Projects\ML\ADHD\Peking_1"

.


In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("..")

from Code.data_generator import FMRIDataGenerator

In [3]:
import numpy as np
import pandas as pd
import os
import sys
from datetime import datetime

import tensorflow as tf

from tensorflow.keras.layers import Conv3D, MaxPool3D, TimeDistributed, Flatten, LSTM, Dense
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import CSVLogger

import tensorflow.keras as keras
from keras.models import load_model

import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [8]:
# ============================ DATA WORK ============================

#file_num = sys.argv[0]
# file_num = "peking"

# Dataframes
model_train_data = pd.read_csv(os.path.join(root, "Data\\training_peking.csv"))
model_val_data = pd.read_csv(os.path.join(root, "Data\\validation_peking.csv"))

# Dictionary of data values
partition = {'train': model_train_data['Image'].values, 
             'validation': model_val_data['Image'].values}

# Training Data
train_labels = {}
for index, row in model_train_data.iterrows():
    train_labels[row['Image']] = row['DX']
    
# Validation Data
val_labels = {}
for index, row in model_val_data.iterrows():
    val_labels[row['Image']] = row['DX']

In [9]:
# ============================ MODEL META ============================

epochs = 5
batch_size = 6
input_shape=(177,28,28,28,1)

train_steps_per_epoch = model_train_data.shape[0] // batch_size
validate_steps_per_epoch = model_val_data.shape[0] // batch_size

# Generators
training_generator = FMRIDataGenerator(partition['train'], train_labels, dataset_dir, batch_size)
validation_generator = FMRIDataGenerator(partition['validation'], val_labels, dataset_dir, batch_size)

curr_time = f'{datetime.now():%H-%M-%S%z_%m%d%Y}'
logger_path = dataset_dir + "/adhd-fmri-history_cv{num}_{time}.csv".format(num=file_num,time=curr_time)

csv_logger = CSVLogger(logger_path, append=True)

callbacks = [csv_logger]

In [10]:
# ============================ MODEL ARCHITECTURE ============================

with tf.device('/gpu:0'):
    cnn_lstm_model = Sequential()

    cnn_lstm_model.add(TimeDistributed(Conv3D(filters=64,kernel_size=(3,3,3),activation='relu'),
                                  input_shape=input_shape, name="Input_Conv_Layer"))

    cnn_lstm_model.add(TimeDistributed(MaxPool3D(
                                    pool_size=(2, 2, 2),
                                    strides=(2, 2, 2),
                                    padding='valid'
                                    ), name="Pool_Layer_1"))

    cnn_lstm_model.add(TimeDistributed(Flatten(), name="Flatten_Layer"))
    
with tf.device('/cpu:0'):

    cnn_lstm_model.add(LSTM(10, dropout = 0.3, recurrent_dropout = 0.3, name="LSTM_Layer"))

with tf.device('/gpu:0'):

    cnn_lstm_model.add(Dense(1, activation = 'sigmoid', name="Output_Dense_Layer"))

    cnn_lstm_model.compile(optimizer=optimizers.Adam(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
cnn_lstm_model.fit_generator(generator=training_generator,
    steps_per_epoch=train_steps_per_epoch, verbose=1, callbacks=callbacks,
    validation_data=validation_generator, validation_steps=validate_steps_per_epoch,
    epochs=epochs)

In [ ]:
cnn_lstm_model.save('my_model2.h5')

In [6]:
cnn_lstm_model = load_model('my_model2.h5')

In [31]:
#PREDICTING A SINGLE FILE

import nibabel as nib
import numpy as np
import random

# filepath for a random nii file
nii_file = random.choice(os.listdir(dataset_dir))

# printing the nii file
print(nii_file, model_train_data[model_train_data['Image'] == nii_file]['DX'].values[0])

# Load the NII file using nibabel                        
img = nib.load(os.path.join(dataset_dir, nii_file))

# Get the image data as a numpy array
data = img.get_fdata()

# Preprocess the image to match the input shape expected by the model
# (e.g., resize, normalize, etc.)
preprocessed_data = training_generator.preprocess_image(nii_file)

# Reshape the preprocessed data to match the input shape expected by the model
reshaped_data = np.reshape(preprocessed_data, input_shape)

# Make the prediction using the model
prediction = cnn_lstm_model.predict(np.expand_dims(reshaped_data, axis=0))

# Process the prediction result as needed
print(prediction)

snwmrda1791543_session_1_rest_1.nii.gz 3
1/1 [==============================] - 2s 2s/step
[[0.4280245]]


In [33]:
#PREDICTING A MULTIPLE FILE

import nibabel as nib
import numpy as np
import random

# filepath for a random nii file
nii_files = random.choices(os.listdir(dataset_dir), k=3)

for nii_file in nii_files:
    # printing the nii file
    print(nii_file, model_train_data[model_train_data['Image'] == nii_file]['DX'].values[0])

    # Load the NII file using nibabel                        
    img = nib.load(os.path.join(dataset_dir, nii_file))

    # Get the image data as a numpy array
    data = img.get_fdata()

    # Preprocess the image to match the input shape expected by the model
    # (e.g., resize, normalize, etc.)
    preprocessed_data = training_generator.preprocess_image(nii_file)

    # Reshape the preprocessed data to match the input shape expected by the model
    reshaped_data = np.reshape(preprocessed_data, input_shape)

    # Make the prediction using the model
    prediction = cnn_lstm_model.predict(np.expand_dims(reshaped_data, axis=0))

    # Process the prediction result as needed
    print(prediction)
    
    prediction = prediction[0][0] * np.max(model_train_data['DX'])
    print(prediction)
    

sfnwmrda2714224_session_1_rest_1.nii.gz 0
1/1 [==============================] - 2s 2s/step
[[0.34515667]]
1.0354700088500977
sfnwmrda4095748_session_1_rest_1.nii.gz 1
1/1 [==============================] - 2s 2s/step
[[0.3039945]]
0.9119835197925568
sfnwmrda1947991_session_1_rest_1.nii.gz 3
1/1 [==============================] - 2s 2s/step
[[0.32547817]]
0.9764344990253448


In [23]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[]


# Misc. 

In [7]:
# # https://stackoverflow.com/a/46216013/9221241
# def get_model_memory_usage(batch_size, model):
#     import numpy as np
#     from keras import backend as K

#     shapes_mem_count = 0
#     internal_model_mem_count = 0
#     for l in model.layers:
#         layer_type = l.__class__.__name__
#         if layer_type == 'Model':
#             internal_model_mem_count += get_model_memory_usage(batch_size, l)
#         single_layer_mem = 1
#         for s in l.output_shape:
#             if s is None:
#                 continue
#             single_layer_mem *= s
#         shapes_mem_count += single_layer_mem

#     trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
#     non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

#     number_size = 4.0
#     if K.floatx() == 'float16':
#          number_size = 2.0
#     if K.floatx() == 'float64':
#          number_size = 8.0

#     total_memory = number_size*(batch_size*shapes_mem_count + trainable_count + non_trainable_count)
#     gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
#     return gbytes

In [9]:
# cnn_lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Conv_Layer (TimeDist  (None, 177, 26, 26, 26,   1792      
 ributed)                     64)                                
                                                                 
 Pool_Layer_1 (TimeDistribu  (None, 177, 13, 13, 13,   0         
 ted)                         64)                                
                                                                 
 Flatten_Layer (TimeDistrib  (None, 177, 140608)       0         
 uted)                                                           
                                                                 
 LSTM_Layer (LSTM)           (None, 10)                5624760   
                                                                 
 Output_Dense_Layer (Dense)  (None, 1)                 11        
                                                      